# All Cells - Last Updated 11/16, 10:15pm, by Sola

# The last three Cells - Last Updated 11/19, 12:55am, by Yining

# Notes for Progress

- Ran it, there is an issue in model_train
- Refer to the error in the cell

# Update by Sola, 11/20/ 12am; updated cells marked with comments

- Issue looks like mismatch between input and label size, how to fix? truncating the input is not working properly (label size is 1, 423 but input is 1, 1000)
- also was issue with new token for our task being added, but that seems fixed

### Notes on the Fixed Error:
/usr/local/src/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1284: indexSelectLargeIndex: block: [36,0,0], thread: [96,0,0] Assertion srcIndex < srcSelectDimSize failed.

- seems to be due to issue with added token "<CoTVMCQA"; helpful source: https://github.com/pytorch/pytorch/issues/121493
- also seems to be due to issue of different input (turned into input_id) and label sizes:
input_ids shape: torch.Size([1, 423])
labels shape: torch.Size([1, 1000]
- current error is caused by an attempt to fix this problem)

## Update on the "Fixed Error" - Sola 11/20/2024, 10:53pm

### I got them to be the same size but I am back to the original error; working slices seem to be running

# Error Has Been Fixed!

## Update 11/22/2024, 5:30pm - Sola 
- trying to improve finetuned model by running more epochs (starting at epoch 2 checkpoint for speed)

## Update 11/23/2024/, 12:35pm -Sola
- Adding 1 more epoch to make it 6

In [1]:
#process based on notebook here: https://huggingface.co/blog/finetune-florence2
!pip install -q datasets flash_attn timm einops 

In [2]:
import torch
from datasets import load_dataset 

working_indices = list(range(1000, 1200)) + list(range(3000, 4000)) + list(range(4000, 4500)) + list(range(5000, 6000))
#load data and split it, currently only has train
data = load_dataset("JiayiHe/SELFCORSET", split="train") #total 7.39k; working 2700...
working_data = data.select(working_indices)
#data = data.train_test_split(test_size=0.2) #5.912k train, 1.478k test
#train_data = data['train']
#half = int(len(train_data) * 0.5)
#print(half)
#initial_fine_tune_data = train_data.select(range(half)) #2.956k for initial fine-tune

#test split data here is validation data
#initial_fine_tune_data = initial_fine_tune_data.train_test_split(test_size=0.3) #2.0692k for train, 0.8868k for valid 
initial_fine_tune_data = working_data.train_test_split(test_size = 0.3)


DPO_CPM.parquet:   0%|          | 0.00/258M [00:00<?, ?B/s]

DPO_LLaVA_13b.parquet:   0%|          | 0.00/90.0M [00:00<?, ?B/s]

DPO_LLaVA_7b.parquet:   0%|          | 0.00/265M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7388 [00:00<?, ? examples/s]

In [3]:
#save splits to ensure we do not reuse data where we shouldn't
florence_fine_tune_train = initial_fine_tune_data['train']
#florence_fine_tune_train.save_to_disk("florence-ft-train-data")

florence_fine_tune_valid = initial_fine_tune_data['test']
#florence_fine_tune_valid.save_to_disk("florence-ft-valid-data")

#rationale_improvement_prompt_and_tune_data = train_data.select(range(half, len(train_data)))
#rationale_improvement_prompt_and_tune_data.save_to_disk("data-for-rationale-fine-tuning")

#test_for_evaluation = data['test']
#test_for_evaluation.save_to_disk("selfcorset-eval-data")

In [4]:
#; Sola Updated, 11/20/2024 12a
from transformers import AutoModelForCausalLM, AutoProcessor, AutoTokenizer, AutoConfig
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#model = AutoModelForCausalLM.from_pretrained("microsoft/Florence-2-base-ft", trust_remote_code=True, revision='refs/pr/6').to(device)
#processor = AutoProcessor.from_pretrained("microsoft/Florence-2-base-ft", trust_remote_code=True, revision='refs/pr/6')

#processor.tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base", use_fast=False)

#issue might be caused by odd tokens/token_embeddings not being updated, so add special token


'''
special_token = "<CoTVMCQA>"
special_tokens_dict = {"additional_special_tokens": [special_token]}
processor.tokenizer.add_special_tokens(special_tokens_dict)

commented out because we should be using special version I think
new_tokens_dict = {"cot_mvqa_token": "<CoTVMCQA>"}
processor.tokenizer.add_tokens(["<CoTVMCQA>"])

model.resize_token_embeddings(len(processor.tokenizer))

model.resize_token_embeddings(len(processor.tokenizer))
print(processor.tokenizer.special_tokens_map)
print(processor.tokenizer.additional_special_tokens)
'''

#weird config issue is fixed by this
config = AutoConfig.from_pretrained('/kaggle/input/florence-2-finetuned-edited/Florence-2-Finetuned-edited/Florence-2-Finetuned-edited/Florence-2-CoTVMCQA_model', trust_remote_code=True)
config.vision_config.model_type = "davit"  # Override the model type

model = AutoModelForCausalLM.from_pretrained('/kaggle/input/florence-2-cotvmcqa-model/Florence-2-CoTVMCQA_model_5_epochs', config = config, device_map="cuda", trust_remote_code=True) #start at epoch 2 to save time
processor = AutoProcessor.from_pretrained('/kaggle/input/florence-2-cotvmcqa-model/Florence-2-CoTVMCQA_processor_5_epochs', device_map="cuda", trust_remote_code=True)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

configuration_florence2.py:   0%|          | 0.00/15.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-base-ft:
- configuration_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
You are using a model of type davit to instantiate a model of type florence2. This is not supported for all configurations of models and can yield errors.


modeling_florence2.py:   0%|          | 0.00/127k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-base-ft:
- modeling_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
Florence2LanguageForConditionalGeneration has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, ot

In [5]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1" #; Sola Updated, 11/20/2024 12am

In [6]:
import io
import base64
from PIL import Image
def decode_base64_to_image(base64_string):
    decoded_string = io.BytesIO(base64.b64decode(base64_string))
    img = Image.open(decoded_string)
    return img

In [7]:
from torch.utils.data import Dataset

class CoTVMCQADataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        example = self.data[idx]
        question = "<CoTVMCQA>" + example['prompt']
        answer = example['response']
        image = decode_base64_to_image(example['image'])
        if image.mode != "RGB":
            image = image.convert("RGB")
        #print(f"Image type: {type(image)}")
        return question, answer, image


In [8]:
# Sola Updated, 11/20/2024 12am
import os
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import (AdamW, AutoProcessor, get_scheduler)

def collate_fn(batch):
    questions, answers, images = zip(*batch)
    #issue seems to be caused by the below line! for some reason it is maxxing at 447?...is this consistent? no
    inputs = processor(text=list(questions), images=list(images), return_tensors="pt", padding = True).to(device) #attempt to resize inputs to be as small as labels 
    return inputs, answers

# Create datasets
train_dataset = CoTVMCQADataset(florence_fine_tune_train)
val_dataset = CoTVMCQADataset(florence_fine_tune_valid)

# Create DataLoader
batch_size = 1 #T4 can only handle one
num_workers = 0

train_loader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=collate_fn, num_workers=num_workers, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, collate_fn=collate_fn, num_workers=num_workers)

In [9]:
# Sola Updated, 11/20/2024 12am
def train_model(train_loader, val_loader, model, processor, epochs=10, lr=1e-6):
    optimizer = AdamW(model.parameters(), lr=lr)
    num_training_steps = epochs * len(train_loader)
    lr_scheduler = get_scheduler(
        name="linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps,
    )

    for epoch in range(epochs):
        model.train()
        train_loss = 0
        i = -1
        for batch in tqdm(train_loader, desc=f"Training Epoch {epoch + 1}/{epochs}"):
            i += 1
            inputs, answers = batch

            input_ids = inputs["input_ids"]
            pixel_values = inputs["pixel_values"] #max_length = input_ids.shape[1], truncation= True,
            labels = processor.tokenizer(text=answers, return_tensors="pt",  padding = True, return_token_type_ids=False).input_ids.to(device)
            
            #print(f"input_ids shape: {input_ids.shape}")
            #print(f"labels shape: {labels.shape}")
            #print(f"Max input_ids: {input_ids.max()}, vocab size: {model.config.vocab_size}")
            
            #1022 is max_pos_embedddings - 2, when use max_length, need padding to be that instead of true
            outputs = model(input_ids=input_ids, pixel_values=pixel_values, labels=labels)
            loss = outputs.loss

            loss.backward()
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()

            train_loss += loss.item()

        avg_train_loss = train_loss / len(train_loader)
        print(f"Average Training Loss: {avg_train_loss}")

        # Validation phase
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch in tqdm(val_loader, desc=f"Validation Epoch {epoch + 1}/{epochs}"):
                inputs, answers = batch

                input_ids = inputs["input_ids"]
                pixel_values = inputs["pixel_values"] #max_length = input_ids.shape[1], truncation= True,
                labels = processor.tokenizer(text=answers, return_tensors="pt",  padding = True, return_token_type_ids=False).input_ids.to(device)

                outputs = model(input_ids=input_ids, pixel_values=pixel_values, labels=labels)
                loss = outputs.loss

                val_loss += loss.item()

        avg_val_loss = val_loss / len(val_loader)
        print(f"Average Validation Loss: {avg_val_loss}")

        # Save model checkpoint
        '''I don't want to overwrite the 3,4,5 checkpoints
        output_dir = f"./model_checkpoints/epoch_{epoch+5}" #only because I am using a model that was already fine-tuned for 2 epochs
        os.makedirs(output_dir, exist_ok=True)
        model.save_pretrained(output_dir)
        processor.save_pretrained(output_dir)
        '''


In [10]:
#can decrease resource usage, but makes it worse, maybe use
#for param in model.vision_tower.parameters():
#  param.is_trainable = False

In [11]:
train_model(train_loader, val_loader, model, processor, epochs=1)


/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Training Epoch 1/1: 100%|██████████| 1890/1890 [21:58<00:00,  1.43it/s]


Average Training Loss: 1.189715575936827


Validation Epoch 1/1: 100%|██████████| 810/810 [02:57<00:00,  4.55it/s]

Average Validation Loss: 1.05575319563763


In [12]:
model.save_pretrained("Florence-2-CoTVMCQA_model_epochs") #6 epoch model! fine-tuned edited has 2 epoch model
processor.save_pretrained("Florence-2-CoTVMCQA_processor_epochs")

['Florence-2-CoTVMCQA_processor_epochs/processor_config.json']